In [1]:
import numpy as np
import random as random
from enum import Enum
import ipywidgets as widgets
import IPython.display as display

class Game:
    Input = Enum('Input', 'up down left right')
    
    def __init__(self, startPos=None):
        if startPos is not None:
            self.board = np.array(startPos)
        else:
            self.board = np.zeros(16, dtype=np.int16).reshape(4,4)
            for i in range(2):
                self.spawn_new_tile()
        self.score = 0
        self.game_over = False

    def run(self):
        btn_left = widgets.Button(description='Left')
        btn_up = widgets.Button(description='Up')
        btn_right = widgets.Button(description='Right')
        btn_down = widgets.Button(description='Down')
        
        def click(btn):
            action = None;
            if btn is btn_left:
                action = Game.Input.left
            elif btn is btn_right:
                action = Game.Input.right
            elif btn is btn_up:
                action = Game.Input.up
            elif btn is btn_down:
                action = Game.Input.down
            self.tick(action)
            
        
        btn_left.on_click(click)
        btn_up.on_click(click)
        btn_down.on_click(click)
        btn_right.on_click(click)
        display.display(widgets.HBox([btn_left, btn_up, btn_right, btn_down]))

    def find_open_tile(self): 
        open_tiles = np.argwhere(self.board == 0)
        return random.choice(open_tiles)
    
    def spawn_new_tile(self):
        index = self.find_open_tile()
        self.board[index[0]][index[1]] = 2 if random.random() < 0.9 else 4
        
    def reward(self, points):
        self.score += points
    
    def checkGameOver(self):
        if not np.any(self.board == 0):
            currentBoard = np.copy(self.board)
            for direction in self.Input:
                simulatedBoard = np.copy(self.board)
                self.slide(simulatedBoard, direction)
                if not np.array_equal(currentBoard, simulatedBoard):
                    return False
            return True
        else:
            return False
    
    def moveTiles(self, transformedBoard):
        def find_first_none_zero(array, start):
            if start is not None:
                for x in range(start, len(array)):
                    if array[x] > 0:
                        return (x, array[x]);
        
        score = 0
        for row in transformedBoard:
            for i in range(len(row)):
                first_non_zero = find_first_none_zero(row, i)
                second_non_zero = find_first_none_zero(row, first_non_zero[0] + 1 if first_non_zero is not None else None)
                if first_non_zero is not None and first_non_zero[0] != i:
                    row[i] = first_non_zero[1]
                    row[first_non_zero[0]] = 0
                if second_non_zero is not None and first_non_zero[1] == second_non_zero[1]:
                    row[i] = first_non_zero[1] + second_non_zero[1]
                    row[second_non_zero[0]] = 0
                    score += row[i]
        return score
    
    def slide(self, board, input):
        if input == Game.Input.up:
            return self.moveTiles(board.T)
        elif input == Game.Input.down:
            return self.moveTiles(np.fliplr(board.T))
        elif input == Game.Input.left:
            return self.moveTiles(board)
        elif input == Game.Input.right:
            return self.moveTiles(np.fliplr(board))

    def tick(self, input):
        if not isinstance(input, Game.Input):
            raise Exception('Illegal input value')
        
        oldBoard = np.copy(self.board)
        score = self.slide(self.board, input)
        if not np.array_equal(oldBoard, self.board):
            self.reward(score)
            self.spawn_new_tile()
            self.game_over = self.checkGameOver()
        
        display.clear_output()
        self.render()
            
                
    
    def render(self):
        print('Score: %d%s' % (self.score, ', GAME OVER!' if self.game_over else ''))
        print(self.board)
        
#game_instance = Game([(2,2,8,16),(8,2,32,128), (2,4,8,16), (128,248,2,8)])
game_instance = Game()
game_instance.render()
game_instance.run()



Score: 0
[[0 0 0 0]
 [2 0 0 0]
 [0 0 0 0]
 [0 0 0 2]]
